#2.TRANSFER LEARNING ALGORİTMALARI KULLANARAK SINIFLANDIRMA YAPMA TRAIN, TEST VE VALIDATION DOĞRULUK ORANLARINI ve TEST VERİSİNİN ACCURACY,PRECISION,RECALL VE F1 SCORE,KARMAŞIKLIK MATRİSİ,CLASSIFICATİON REPORTUNU GÖSTERME

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score

In [3]:
print(os.listdir("/gdrive/MyDrive/Kidney_Cancer/Kidney Cancer"))

DATADIR = '/gdrive/MyDrive/Kidney_Cancer/Kidney Cancer'

['Tumor', 'Normal']


In [4]:
CATEGORIES = ["Tumor", "Normal"]
IMG_SIZE = 100

In [5]:
from keras.applications.vgg16 import preprocess_input
all_data_transfer = []
def create_all_data_transfer():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                # Görüntüyü ön işle
                new_array = preprocess_input(new_array)
                all_data_transfer.append([new_array, class_num])
            except Exception as e:
                pass
create_all_data_transfer()

In [6]:
print("Toplam Görüntü Sayısı: ", len(all_data_transfer))


Toplam Görüntü Sayısı:  10000


In [7]:
X_transfer = []
y_transfer = []

for categories, label in all_data_transfer:
  X_transfer.append(categories)
  y_transfer.append(label)


In [8]:
x_train_transfer, x_test_transfer, y_train_transfer, y_test_transfer = train_test_split(X_transfer, y_transfer, test_size=0.35, random_state=42)

x_train_transfer, x_val_transfer, y_train_transfer, y_val_transfer = train_test_split(x_train_transfer, y_train_transfer, test_size=0.10, random_state=42)

In [9]:
x_train_transfer = np.array(x_train_transfer)
x_test_transfer = np.array(x_test_transfer)

y_train_transfer = np.array(y_train_transfer)
y_test_transfer = np.array(y_test_transfer)

x_val_transfer = np.array(x_val_transfer)
y_val_transfer = np.array(y_val_transfer)

In [10]:
print(x_train_transfer.shape)
print(x_test_transfer.shape)
print(x_val_transfer.shape)

print(y_train_transfer.shape)
print(y_test_transfer.shape)
print(y_val_transfer.shape)

(5850, 100, 100, 3)
(3500, 100, 100, 3)
(650, 100, 100, 3)
(5850,)
(3500,)
(650,)


In [11]:
traitrain_yCl_transfer = tf.keras.utils.to_categorical(y_train_transfer, num_classes=2)
test_yCl_transfer = tf.keras.utils.to_categorical(y_test_transfer, num_classes=2)
valid_yCl_transfer = tf.keras.utils.to_categorical(y_val_transfer, num_classes=2)

In [12]:
# VGG16 modelini yükleyip, fully-connected katmanlarını çıkartma
base_model_transfer = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [13]:
# Fully-connected katmanlarını ekleyerek transfer learning modelini oluşturma
x_transfer = Flatten()(base_model_transfer.output)
x_transfer = Dense(1024, activation='relu')(x_transfer)
x_transfer = Dropout(0.4)(x_transfer)
x_transfer = Dense(1024, activation='relu')(x_transfer)
predictions_transfer = Dense(2, activation='softmax')(x_transfer)

model_transfer_learning = Model(inputs=base_model_transfer.input, outputs=predictions_transfer)

In [14]:
# Önceden eğitilmiş ağırlıkları dondurma
for layer in base_model_transfer.layers:
    layer.trainable = False


In [15]:
# Modeli derleme
model_transfer_learning.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
#Modeli eğitme
history_transfer_learning = model_transfer_learning.fit(x_train_transfer, traitrain_yCl_transfer,
                                                        batch_size=64,
                                                        validation_data=(x_val_transfer, valid_yCl_transfer),
                                                        epochs=30)


Epoch 1/30
92/92 [==============================] - 826s 9s/step - loss: 1.7790 - accuracy: 0.9241 - val_loss: 0.5626 - val_accuracy: 0.9677
Epoch 2/30
92/92 [==============================] - 809s 9s/step - loss: 0.2730 - accuracy: 0.9803 - val_loss: 0.1659 - val_accuracy: 0.9815
Epoch 3/30
92/92 [==============================] - 810s 9s/step - loss: 0.1463 - accuracy: 0.9874 - val_loss: 0.0606 - val_accuracy: 0.9923
Epoch 4/30
92/92 [==============================] - 870s 9s/step - loss: 0.0692 - accuracy: 0.9921 - val_loss: 0.1715 - val_accuracy: 0.9923
Epoch 5/30
92/92 [==============================] - 812s 9s/step - loss: 0.0693 - accuracy: 0.9915 - val_loss: 0.1799 - val_accuracy: 0.9908
Epoch 6/30
92/92 [==============================] - 809s 9s/step - loss: 0.1291 - accuracy: 0.9921 - val_loss: 7.0725e-04 - val_accuracy: 1.0000
Epoch 7/30
92/92 [==============================] - 809s 9s/step - loss: 0.0487 - accuracy: 0.9945 - val_loss: 4.6158e-04 - val_accuracy: 1.0000
Epoch

In [17]:
# Validation veri seti için preprocess_input işlemi
x_val_transfer_preprocessed = preprocess_input(x_val_transfer)
# Test veri seti için preprocess_input işlemi
x_test_transfer_preprocessed = preprocess_input(x_test_transfer)

In [18]:
# Test verisinde performansı değerlendirme
y_pred_transfer = model_transfer_learning.predict(x_test_transfer_preprocessed)
y_pred_classes_transfer = np.argmax(y_pred_transfer, axis=1)
y_test_classes_transfer = np.argmax(test_yCl_transfer, axis=1)

110/110 [==============================] - 428s 4s/step


In [19]:
# Transfer Learning modelini değerlendirme
score_transfer_learning_val = model_transfer_learning.evaluate(x_val_transfer_preprocessed, valid_yCl_transfer)
print("Validation Accuracy (Transfer Learning): ", score_transfer_learning_val[1])

score_transfer_learning_test = model_transfer_learning.evaluate(x_test_transfer_preprocessed, test_yCl_transfer)
print("Test Accuracy (Transfer Learning): ", score_transfer_learning_test[1])

score_transfer_learning_train = model_transfer_learning.evaluate(x_train_transfer, traitrain_yCl_transfer)
print("Train Accuracy (Transfer Learning): ", score_transfer_learning_train[1])


21/21 [==============================] - 79s 4s/step - loss: 0.2790 - accuracy: 0.9769
Validation Accuracy (Transfer Learning):  0.9769230484962463
110/110 [==============================] - 425s 4s/step - loss: 0.2241 - accuracy: 0.9843
Test Accuracy (Transfer Learning):  0.9842857122421265
183/183 [==============================] - 712s 4s/step - loss: 0.0011 - accuracy: 0.9998
Train Accuracy (Transfer Learning):  0.9998290538787842


In [20]:
# Test verisinin Accuracy, Precision, Recall ve F1 skoru, Karmaşıklık Matrisi, Classification Report'u
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Test verisi üzerinde tahmin yapma
y_pred_transfer_learning = model_transfer_learning.predict(x_test_transfer_preprocessed)
y_pred_classes_transfer_learning = np.argmax(y_pred_transfer_learning, axis=1)

# One-hot encoding'i geri çevirme
y_test_classes_transfer_learning = np.argmax(test_yCl_transfer, axis=1)

# Accuracy
accuracy_transfer_learning = accuracy_score(y_test_classes_transfer_learning, y_pred_classes_transfer_learning)
print("Transfer Learning Test Accuracy: {:.2f}%".format(accuracy_transfer_learning * 100))

# Precision
precision_transfer_learning = precision_score(y_test_classes_transfer_learning, y_pred_classes_transfer_learning, average='weighted')
print("Transfer Learning Test Precision: {:.2f}".format(precision_transfer_learning))

# Recall
recall_transfer_learning = recall_score(y_test_classes_transfer_learning, y_pred_classes_transfer_learning, average='weighted')
print("Transfer Learning Test Recall: {:.2f}".format(recall_transfer_learning))

# F1 Score
f1_transfer_learning = f1_score(y_test_classes_transfer_learning, y_pred_classes_transfer_learning, average='weighted')
print("Transfer Learning Test F1 Score: {:.2f}".format(f1_transfer_learning))

# Karmaşıklık Matrisi
conf_matrix_transfer_learning = confusion_matrix(y_test_classes_transfer_learning, y_pred_classes_transfer_learning)
print("Transfer Learning Confusion Matrix:\n", conf_matrix_transfer_learning)

# Classification Report
class_report_transfer_learning = classification_report(y_test_classes_transfer_learning, y_pred_classes_transfer_learning)
print("Transfer Learning Classification Report:\n", class_report_transfer_learning)

110/110 [==============================] - 430s 4s/step
Transfer Learning Test Accuracy: 98.43%
Transfer Learning Test Precision: 0.98
Transfer Learning Test Recall: 0.98
Transfer Learning Test F1 Score: 0.98
Transfer Learning Confusion Matrix:
 [[1778    8]
 [  47 1667]]
Transfer Learning Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      1786
           1       1.00      0.97      0.98      1714

    accuracy                           0.98      3500
   macro avg       0.98      0.98      0.98      3500
weighted avg       0.98      0.98      0.98      3500

